# Security analysis

This notebook will showcase a few security analysis tools which can be used to see how specified contingencies affect powerflow, including seeing which operational limits are violated and how bus and branch parameters change. We begin by initializing the same grid we saw in the earlier powerflow tutorial.

In [ ]:
import pypowsybl as pp
import pandas as pd
from pathlib import Path
import numpy as np

grid = pp.network.load(file='data/recollement-auto-20210101-0000-enrichi.xiidm.bz2')

df_inj_gen = pd.read_parquet("data/snapshot_gen_2021-01-01T00-00-00.parquet", engine="pyarrow")
df_inj_load = pd.read_parquet("data/snapshot_load_2021-01-01T00-00-00.parquet", engine="pyarrow")
df_inj_bus = pd.read_parquet("data/snapshot_bus_2021-01-01T00-00-00.parquet", engine="pyarrow")

df_inj_gen['target_p'] =  df_inj_gen['target_p'] *100

df_gens = grid.get_generators(attributes=['target_p']) 
df_gens.update(df_inj_gen.set_index('id'))
grid.update_generators(df=df_gens)

load_power_factor = 0.98 

df_inj_load['p0'] = df_inj_load['p0']
df_loads = grid.get_loads(attributes=['p0'])
df_loads.update(df_inj_load.set_index('id'))
grid.update_loads(df=df_loads)

df_inj_load['q0'] = df_inj_load['p0'] * np.sqrt( (1-load_power_factor**2)/load_power_factor)
df_loads = grid.get_loads(attributes=['q0'])
df_loads.update(df_inj_load.set_index('id'))
grid.update_loads(df=df_loads)

gen_power_factor = 0.8

# PQ generators
df_inj_pqgens = df_inj_gen.loc[df_inj_gen['voltage_regulator_on'] == False]
df_inj_pqgens['target_q'] = df_inj_pqgens['target_p']* np.sqrt( (1-gen_power_factor**2)/gen_power_factor)
df_pqgens = grid.get_generators(attributes=['target_q']) 
df_pqgens.update(df_inj_pqgens.set_index('id'))
grid.update_generators(df=df_pqgens)

# PV generators
df_inj_pvgens = df_inj_gen.loc[df_inj_gen['voltage_regulator_on'] == True]

lookup_v = df_inj_bus.drop_duplicates(subset='voltage_level_id').set_index('voltage_level_id')['nominal_v'] # Get nominal_v for each voltage_level_id
df_inj_pvgens['target_v'] = df_inj_pvgens['voltage_level_id'].map(lookup_v)
df_pvgens = grid.get_generators(attributes=['target_v'])
df_pvgens.update(df_inj_pvgens.set_index('id'))
grid.update_generators(df=df_pvgens)

df_dlines = grid.get_dangling_lines()

mismatch = grid.get_generators()['target_p'].sum() - grid.get_loads()['p0'].sum()

df_dlines['weight'] = 1.0 / df_dlines['x'].replace(0, np.nan) 
df_dlines['p0'] = mismatch * df_dlines['weight'] / df_dlines['weight'].sum()              
df_dlines['q0'] = 0.0

grid.update_dangling_lines(df=df_dlines[['p0', 'q0']])

ac_params = pp.loadflow.Parameters(
    voltage_init_mode=pp.loadflow.VoltageInitMode.DC_VALUES, # Initialize voltages as 1pu and voltage angles using a DC powerflow
    distributed_slack=True, # The active power mismatch will be distributed over the network
    balance_type=pp.loadflow.BalanceType.PROPORTIONAL_TO_GENERATION_P, # The participation factors for slack distribution are calculated using 'target_p'
    transformer_voltage_control_on=False, # Transformer voltage regulating will not be allowed 
    shunt_compensator_voltage_control_on=False, # Shunt compensator voltage regulating will not be allowed 

    provider_parameters={ 
        'maxNewtonRaphsonIterations': '200',
        'maxOuterLoopIterations': '50',
        'stateVectorScalingMode': 'MAX_VOLTAGE_CHANGE',  # Limits voltage vector updates
        'maxVoltageChangeStateVectorScalingMaxDv': '0.05',  # Limits maximum voltage magnitude update amount
        'maxVoltageChangeStateVectorScalingMaxDphi': '0.08726',  # Limits maximum voltage angle update amount
        'maxSlackBusCount': '10', 
        'newtonRaphsonConvEpsPerEq': '1e-1', # Stopping criterion
        'slackBusPMaxMismatch': '500.0', # Slack power is considered to be distributed when below 500MW
    }
)

## Running the analysis

To run a security analysis we need a network (grid loaded above) and at least one contingency on the network. In the result of the analysis, there is the pre-contingency part, which contains the operational limits exceeded for the grid before the contingency, and the post contingency part, which contains the limits exceeded after.

In [2]:
security_analysis = pp.security.create_analysis()
security_analysis.add_single_element_contingency('BOUCH.TG7', 'Contingency 1') # 585MW thermal plant N-1 contingency
security_analysis.add_single_element_contingency('SSAL77G1', 'Contingency 2') # 1.335GW nuclear plant N-1 contingency
security_analysis.add_single_element_contingencies(('WARANY763', 'WARANY764'), contingency_id_provider= None) # Two separate transformer N-1 contingencies
security_analysis.add_multiple_elements_contingency(('CHIN27CHIN21','WARANY763'), 'Contingency 3') # 905MW nuclear plant and transformer N-2 contingency

result= security_analysis.run_ac(grid, parameters = ac_params)

print("Pre contingency: "+ str(result.pre_contingency_result))
print("Post contingency: "+ str(result.post_contingency_results))

Pre contingency: PreContingencyResult(, status=CONVERGED, limit_violations=[2130])
Post contingency: {'Contingency 1': PostContingencyResult(contingency_id='Contingency 1', status=CONVERGED, limit_violations=[2210]), 'Contingency 2': PostContingencyResult(contingency_id='Contingency 2', status=FAILED, limit_violations=[0]), 'Contingency 3': PostContingencyResult(contingency_id='Contingency 3', status=CONVERGED, limit_violations=[2337]), 'WARANY763': PostContingencyResult(contingency_id='WARANY763', status=CONVERGED, limit_violations=[1965]), 'WARANY764': PostContingencyResult(contingency_id='WARANY764', status=CONVERGED, limit_violations=[1967])}


Reading the above, we see that there is a large number (2130) of limit violations even before applying a contingency. This could be improved if we had synthesized better injection data. We note that:
- Post "Contingency 1", we see that the number of limit violations goes up by 80
- Post "Contingency 2" our powerflow fails to converge
- Post "Contingency 3" the number of limit violations goes up by 207
- After the two transformer contingencies, "WARANY763" and WARANY764", there are fewer violations than there were in the original powerflow (165 and 163 fewer, respectively)

It is worth looking into how the limit violations are stored in `result`.

In [3]:
violations = result.limit_violations
violations

subject_name   limit_type limit_name  \
contingency_id subject_id                                           
               CORNIL61PRESS                   CURRENT  permanent   
               CORNIL61PRESS                   CURRENT       IT10   
               HOURAL31ZESPA                   CURRENT  permanent   
               HOURAL31ZESPA                   CURRENT  permanent   
               ARRIGY632                       CURRENT  permanent   
...                                   ...          ...        ...   
WARANY764      ZVLENP3                     LOW_VOLTAGE              
               ZVLETP3                     LOW_VOLTAGE              
               ZVONNP3                     LOW_VOLTAGE              
               ZY.EVP4                     LOW_VOLTAGE              
               ZYZEUP3                     LOW_VOLTAGE              

                                    limit  acceptable_duration  \
contingency_id subject_id                                        
               CORNIL61PRESS  1168.000000           2147483647   
               CORNIL61PRESS  1517.000000                   60   
               HOURAL31ZESPA   152.000000           2147483647   
               HOURAL31ZESPA   152.000000           2147483647   
               ARRIGY632      1143.000000                 1200   
...                                   ...                  ...   
WARANY764      ZVLENP3          59.000000           2147483647   
               ZVLETP3          59.000000           2147483647   
               ZVONNP3          59.000000           2147483647   
               ZY.EVP4          85.000008           2147483647   
               ZYZEUP3          59.000000           2147483647   

                              limit_reduction        value side  
contingency_id subject_id                                        
               CORNIL61PRESS              1.0  2631.544415  ONE  
               CORNIL61PRESS              1.0  2633.938544  TWO  
               HOURAL31ZESPA              1.0   336.389375  ONE  
               HOURAL31ZESPA              1.0   336.284544  TWO  
               ARRIGY632                  1.0  1287.666532  ONE  
...                                       ...          ...  ...  
WARANY764      ZVLENP3                    1.0    56.276605       
               ZVLETP3                    1.0    58.010725       
               ZVONNP3                    1.0    58.992232       
               ZY.EVP4                    1.0    79.869435       
               ZYZEUP3                    1.0    58.295022       

[10609 rows x 8 columns]

Looking at the above, we may understand that `result.limit_violations` contains the violations before and after each contingency (with the exception of 'Contingency 2', which failed to converge). This can be seen more clearly below, where we limit to elements starting with 'CAMP' and can see the differences between the entries under each contingency.

In [4]:
filtered = violations[
    violations.index
    .get_level_values('subject_id')
    .str.startswith('CAMP')
]
filtered

subject_name   limit_type limit_name  limit  \
contingency_id subject_id                                               
               CAMP P3                  LOW_VOLTAGE              59.0   
Contingency 1  CAMP P3                  LOW_VOLTAGE              59.0   
               CAMP6P3                  LOW_VOLTAGE              59.0   
Contingency 3  CAMP P3                  LOW_VOLTAGE              59.0   
               CAMP6P3                  LOW_VOLTAGE              59.0   
WARANY763      CAMP P3                  LOW_VOLTAGE              59.0   
WARANY764      CAMP P3                  LOW_VOLTAGE              59.0   

                           acceptable_duration  limit_reduction      value  \
contingency_id subject_id                                                    
               CAMP P3              2147483647              1.0  58.531925   
Contingency 1  CAMP P3              2147483647              1.0  58.224334   
               CAMP6P3              2147483647              1.0  58.357076   
Contingency 3  CAMP P3              2147483647              1.0  58.163608   
               CAMP6P3              2147483647              1.0  57.938470   
WARANY763      CAMP P3              2147483647              1.0  58.482975   
WARANY764      CAMP P3              2147483647              1.0  58.482822   

                          side  
contingency_id subject_id       
               CAMP P3          
Contingency 1  CAMP P3          
               CAMP6P3          
Contingency 3  CAMP P3          
               CAMP6P3          
WARANY763      CAMP P3          
WARANY764      CAMP P3

We can also get a JSON file with the full security analysis results:

In [5]:
result.export_to_json(str('json_file.json'))

We can extract information about chosen voltage levels and branches before and after contingecies by monitoring them. In this way we can see how bus voltages and branch currents & impedances change with different contingency configurations. Below we run a new security analysis to showcase this capability 

In [6]:
security_analysis2 = pp.security.create_analysis()
security_analysis2.add_single_element_contingency('BOUCH.TG7', 'Contingency A') # 585MW thermal plant N-1 contingency
security_analysis2.add_single_element_contingency('CHIN27CHIN21', 'Contingency B') # 905MW nuclear plant N-1 contingency
security_analysis2.add_single_element_contingency('CHIN2L71G.AVO', 'Contingency C') # One of two branches connected to nuclear plant voltage level
security_analysis2.add_monitored_elements(voltage_level_ids=['CHIN2P7','SSAL7P7']) # Voltage levels of the above nuclear plant and another one of a 1.335GW
security_analysis2.add_monitored_elements(branch_ids=['CHIN2L71G.AVO','CHIN2L72G.AVO']) # Branches attached to nuclear plant voltage level 'CHIN2P7'

# For if you only want to see the precontingency rows of the monitored elements
# security_analysis2.add_precontingency_monitored_elements(voltage_level_ids=['CHIN2P7','SSAL7P7']) # For if you only want to see the precontingency rows of the monitored elements

# For if you only want to see the postcontingency rows of the monitored elements (you may specify which contingencies)
# security_analysis2.add_postcontingency_monitored_elements(voltage_level_ids=['CHIN2P7','SSAL7P7'], contingency_ids=['Contingency A','Contingency B'])

result2= security_analysis2.run_ac(grid, parameters = ac_params)

result2.bus_results

v_mag  \
contingency_id operator_strategy_id voltage_level_id bus_id                  
                                    CHIN2P7          CHIN2P7_3  380.000000   
                                                     CHIN2P7_0  380.000000   
                                                     CHIN2P7_5  380.000000   
                                                     CHIN2P7_1  380.000000   
                                    SSAL7P7          SSAL7P7_0  380.000000   
                                                     SSAL7P7_7  380.000000   
                                                     SSAL7P7_1  380.000000   
                                                     SSAL7P7_9  380.000000   
Contingency A                       CHIN2P7          CHIN2P7_3  380.000000   
                                                     CHIN2P7_0  380.000000   
                                                     CHIN2P7_5  380.000000   
                                                     CHIN2P7_1  380.000000   
                                    SSAL7P7          SSAL7P7_0  380.000000   
                                                     SSAL7P7_7  380.000000   
                                                     SSAL7P7_1  380.000000   
                                                     SSAL7P7_9  380.000000   
Contingency B                       CHIN2P7          CHIN2P7_3  379.989452   
                                                     CHIN2P7_0  379.989452   
                                                     CHIN2P7_5  380.000000   
                                                     CHIN2P7_1  380.000000   
                                    SSAL7P7          SSAL7P7_0  380.000000   
                                                     SSAL7P7_7  380.000000   
                                                     SSAL7P7_1  380.000000   
                                                     SSAL7P7_9  380.000000   
Contingency C                       CHIN2P7          CHIN2P7_5  380.000000   
                                                     CHIN2P7_1  380.000000   
                                    SSAL7P7          SSAL7P7_0  380.000000   
                                                     SSAL7P7_7  380.000000   
                                                     SSAL7P7_1  380.000000   
                                                     SSAL7P7_9  380.000000   

                                                                  v_angle  
contingency_id operator_strategy_id voltage_level_id bus_id                
                                    CHIN2P7          CHIN2P7_3 -10.796081  
                                                     CHIN2P7_0 -10.796081  
                                                     CHIN2P7_5 -10.792507  
                                                     CHIN2P7_1 -10.792507  
                                    SSAL7P7          SSAL7P7_0   5.272102  
                                                     SSAL7P7_7   5.272102  
                                                     SSAL7P7_1   5.278919  
                                                     SSAL7P7_9   5.278919  
Contingency A                       CHIN2P7          CHIN2P7_3 -10.905573  
                                                     CHIN2P7_0 -10.905573  
                                                     CHIN2P7_5 -10.901993  
                                                     CHIN2P7_1 -10.901993  
                                    SSAL7P7          SSAL7P7_0   5.250187  
                                                     SSAL7P7_7   5.250187  
                                                     SSAL7P7_1   5.257008  
                                                     SSAL7P7_9   5.257008  
Contingency B                       CHIN2P7          CHIN2P7_3 -13.375480  
                                                     CHIN2P7_0 -13.375480  
                                                    

We can see from the voltage level monitoring above, and the branch monitoring below, how we may track changes that occur in our grid when an element fails.

In [7]:
result2.branch_results

p1         q1  \
contingency_id operator_strategy_id branch_id                              
                                    CHIN2L72G.AVO  893.787829 -22.512301   
                                    CHIN2L71G.AVO  893.842909 -18.636715   
Contingency A                       CHIN2L71G.AVO  894.779575 -20.541609   
                                    CHIN2L72G.AVO  894.737863 -24.324586   
Contingency B                       CHIN2L71G.AVO  557.060052 -12.218677   
                                    CHIN2L72G.AVO  895.250742 -28.645573   
Contingency C                       CHIN2L72G.AVO  895.270760 -27.327778   

                                                            i1          p2  \
contingency_id operator_strategy_id branch_id                                
                                    CHIN2L72G.AVO  1358.400802 -893.705575   
                                    CHIN2L71G.AVO  1358.348959 -893.763375   
Contingency A                       CHIN2L71G.AVO  1359.835113 -894.699867   
                                    CHIN2L72G.AVO  1359.915816 -894.655426   
Contingency B                       CHIN2L71G.AVO   846.592258 -557.029158   
                                    CHIN2L72G.AVO  1360.888902 -895.168186   
Contingency C                       CHIN2L72G.AVO  1360.856743 -895.188209   

                                                          q2           i2  \
contingency_id operator_strategy_id branch_id                               
                                    CHIN2L72G.AVO  23.260061  1358.398960   
                                    CHIN2L71G.AVO  19.382091  1358.347345   
Contingency A                       CHIN2L71G.AVO  21.288726  1359.833339   
                                    CHIN2L72G.AVO  25.074120  1359.913831   
Contingency B                       CHIN2L71G.AVO  12.477661   846.590576   
                                    CHIN2L72G.AVO  29.396247  1360.886571   
Contingency C                       CHIN2L72G.AVO  28.078415  1360.854517   

                                                   flow_transfer  
contingency_id operator_strategy_id branch_id                     
                                    CHIN2L72G.AVO            NaN  
                                    CHIN2L71G.AVO            NaN  
Contingency A                       CHIN2L71G.AVO            NaN  
                                    CHIN2L72G.AVO            NaN  
Contingency B                       CHIN2L71G.AVO            NaN  
                                    CHIN2L72G.AVO            NaN  
Contingency C                       CHIN2L72G.AVO       0.001659

## Operator strategies and remedial actions

In a security analysis we can also define operator strategies and associated remedial actions. The actions can be defined using the `add_`(type)`_action()` API. The types of actions are:
- `switch`, to open/close a switch
- `phase_tap_changer_position`, to change the tap position on a phase tap changer
- `ratio_tap_changer_position`, to change the tap position on a ratio tap changer
- `load_active_power`, to change the active power of a load
- `load_reactive_power`, to change the reactive power of a load
- `shunt_compensator_position`, to change the section of a shunt compensator
- `generator_active_power`, to change the active power of a generator
- `terminals_connection`, to connect/disconnect one or multiple sides of a network element



In the case of N-1 branch contingencies, like Contingency C, we note that the monitored branches also have an associated `flow_transfer` which quantifies how much power is rerouted through them, by $$F_i = \frac{P_i^{post}-P_i^{pre}}{P_{cont}^{pre}} = \frac{895.270760-893.787829}{893.842909} = 0.001659$$ where $F_i$ is known as the line outage distribution factor for the line $i$, and $P_{cont}^{pre}$ is the power of the line which fails (pre-contingency) .


In [8]:
security_analysis3 = pp.security.create_analysis()
security_analysis3.add_single_element_contingency('ARGOEIN3', 'Contingency i') # Wind generator contingency
security_analysis3.add_single_element_contingency('CHIN27CHIN21', 'Contingency ii') # 905MW nuclear plant contingency
security_analysis3.add_single_element_contingency('ARGOEP4_ARGOE   4TR412    DJ.BT.ARGOEIN3', 'Contingency iii') # Breaker switch contingency
security_analysis3.add_multiple_elements_contingency(('ARGOEIN3', 'CHIN27CHIN21'), 'Contingency i+ii') # Both contingencies
security_analysis3.add_monitored_elements(branch_ids=['ARGOEL41V.MAR','ARGOEY741']) # Branches connected to wind generator node
security_analysis3.add_monitored_elements(voltage_level_ids=['ARGOEP4']) # Voltage level 'ARGOEP4'
security_analysis3.add_switch_action(action_id='SwitchAction', switch_id='ARGOEP4_ARGOE   4TR412    DJ.BT.ARGOEIN3', open=True) # Action: breaker switch opens
security_analysis3.add_generator_active_power_action(action_id='GenAction', generator_id= 'BOUCH.TG7', is_relative = True ,active_power = 0.1 ) # Action: generator drops to 0.1 of prior power
security_analysis3.add_operator_strategy(operator_strategy_id='Operator Strategy i', contingency_id='Contingency i', action_ids=['SwitchAction'], condition_type= pp.security.ConditionType.TRUE_CONDITION)
security_analysis3.add_operator_strategy(operator_strategy_id='Operator Strategy ii', contingency_id='Contingency ii', action_ids=['GenAction'], condition_type= pp.security.ConditionType.TRUE_CONDITION)
# security_analysis3.add_operator_strategy(operator_strategy_id='Operator Strategy i+ii', contingency_id='Contingency i+ii', action_ids=['SwitchAction','GenAction'], condition_type= pp.security.ConditionType.TRUE_CONDITION)
result3 = security_analysis3.run_ac(grid, parameters=ac_params)
print(result3.post_contingency_results)
result3.branch_results

{'Contingency i': PostContingencyResult(contingency_id='Contingency i', status=CONVERGED, limit_violations=[2176]), 'Contingency i+ii': PostContingencyResult(contingency_id='Contingency i+ii', status=CONVERGED, limit_violations=[2451]), 'Contingency ii': PostContingencyResult(contingency_id='Contingency ii', status=CONVERGED, limit_violations=[2401]), 'Contingency iii': PostContingencyResult(contingency_id='Contingency iii', status=CONVERGED, limit_violations=[2176])}


p1         q1  \
contingency_id   operator_strategy_id branch_id                             
                                      ARGOEL41V.MAR -63.902914 -72.056402   
                                      ARGOEY741      17.677431  39.677806   
Contingency i                         ARGOEL41V.MAR -64.155137 -71.965371   
                                      ARGOEY741      17.128213  42.485018   
Contingency i+ii                      ARGOEL41V.MAR -64.416581 -71.887447   
                                      ARGOEY741      16.514266  45.355136   
Contingency ii                        ARGOEL41V.MAR -63.855063 -71.696544   
                                      ARGOEY741      17.445880  45.315207   
Contingency iii                       ARGOEL41V.MAR -64.155137 -71.965372   
                                      ARGOEY741      17.128214  42.485015   
Contingency i    Operator Strategy i  ARGOEL41V.MAR -64.279536 -71.929492   
                                      ARGOEY741      16.767483  43.898834   
Contingency ii   Operator Strategy ii ARGOEL41V.MAR -63.843899 -71.588647   
                                      ARGOEY741      17.405478  46.953883   

                                                             i1         p2  \
contingency_id   operator_strategy_id branch_id                              
                                      ARGOEL41V.MAR  601.276355  64.350793   
                                      ARGOEY741      271.185104 -17.622895   
Contingency i                         ARGOEL41V.MAR  599.801367  64.600758   
                                      ARGOEY741      284.986566 -17.067984   
Contingency i+ii                      ARGOEL41V.MAR  598.643939  64.860434   
                                      ARGOEY741      299.353071 -16.447811   
Contingency ii                        ARGOEL41V.MAR  595.366337  64.293918   
                                      ARGOEY741      301.109931 -17.378643   
Contingency iii                       ARGOEL41V.MAR  599.801367  64.600757   
                                      ARGOEY741      284.986553 -17.067985   
Contingency i    Operator Strategy i  ARGOEL41V.MAR  599.323024  64.724426   
                                      ARGOEY741      291.951635 -16.704274   
Contingency ii   Operator Strategy ii ARGOEL41V.MAR  593.576165  64.280038   
                                      ARGOEY741      309.877564 -17.334269   

                                                            q2          i2  \
contingency_id   operator_strategy_id branch_id                              
                                      ARGOEL41V.MAR  70.870381  582.383953   
                                      ARGOEY741     -36.912701   63.139523   
Contingency i                         ARGOEL41V.MAR  70.736392  580.890898   
                                      ARGOEY741     -39.431301   66.352891   
Contingency i+ii                      ARGOEL41V.MAR  70.621408  579.721446   
                                      ARGOEY741     -41.985775   69.697818   
Contingency ii                        ARGOEL41V.MAR  70.397095  576.391654   
                                      ARGOEY741     -41.906182   70.106864   
Contingency iii                       ARGOEL41V.MAR  70.736392  580.890898   
                                      ARGOEY741     -39.431298   66.352888   
Contingency i    Operator Strategy i  ARGOEL41V.MAR  70.684542  580.408677   
                                      ARGOEY741     -40.694027   67.974555   
Contingency ii   Operator Strategy ii ARGOEL41V.MAR  70.254395  574.575985   
                                      ARGOEY741     -43.343442   72.148216   

                                                     flow_transfer  
contingency_id   operator_strategy_id branch_id                     
                                      ARGOEL41V.MAR            NaN  
                                      ARGOEY741                NaN  
Contingency i                         ARGOEL41V.MAR  

We notice a few things from output above:
- All of the contingencies, including with and without the operator strategies where relevant, are present.
- Operator Strategy i does not activate in Contingency i+ii, because it is the `contingency_id` which matters, not the elements in the contingency
- If we wish for the remedial actions of Operator Strategies i and ii to apply in Contingency i+ii, we must add an operator strategy like the commented one in the script above